## 0. 환경 설정하기

### 1) 구글 드라이브 연결하기

In [ ]:
# 코랩 사용 시 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2) 경로 확인하기
- "WORK_SPACE" 에 본인 작업 경로 작성 후 실행(구글 드라이브 최상위에 압축해제 시 그대로 실행. 수정 X).<br>

In [ ]:
# ROOT_PATH 확인 
import os

# 구글 드라이브 내 프로젝트 압축해제된 영역 (구글 드라이브 최상위에 압축해제 시 그대로 실행 수정 X)
WORK_SPACE = "KT_AIVLE"

if os.getcwd() == '/content' :
  # 구글 드라이브 사용 시 
  ROOT_PATH = "/content/drive/MyDrive/"+WORK_SPACE+"/AIVLE3rd_individual"
else :
  ROOT_PATH = os.path.abspath('..')
# Train 데이터 셋 경로
TRAIN_PATH = ROOT_PATH + "/train"
# MODEL 저장 경로
MODEL_PATH = ROOT_PATH + "/model"

### 3) 라이브러리 불러오기

In [ ]:
# 필요 라이브러리 불러오기.
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping

### 4) GPU 환경 확인하기


In [ ]:
# GPU 환경 확인하기
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7320023453197468865
 xla_global_id: -1]

---

In [ ]:
# read video list

video_path = ROOT_PATH + '/video'
file_list = os.listdir(video_path)
# os.makedirs(path)

file_list

['jung.mp4', 'park.mp4', 'sam.mp4', 'sayuri.mp4']

In [ ]:
import cv2

for i in file_list:
    
    video_file =  video_path + '/' + i
    
    shuttleVideo = cv2.VideoCapture(video_file)
    
    name = i[:-4]
    
    path = ROOT_PATH + "/image/" + name

    print(path)

/content/drive/MyDrive/KT_AIVLE/AIVLE3rd_individual/image/jung
/content/drive/MyDrive/KT_AIVLE/AIVLE3rd_individual/image/park
/content/drive/MyDrive/KT_AIVLE/AIVLE3rd_individual/image/sam
/content/drive/MyDrive/KT_AIVLE/AIVLE3rd_individual/image/sayuri


In [ ]:
video_file

'/content/drive/MyDrive/KT_AIVLE/AIVLE3rd_individual/video/sayuri.mp4'

In [ ]:
import cv2

for i in file_list:
    
    video_file =  video_path + '/' + i
    
    shuttleVideo = cv2.VideoCapture(video_file)
    
    name = i[:-4]
    
    path = ROOT_PATH + "/image/" + name
    #os.makedirs(path)
    
    #img_gray_all = []
    
    c1 = 0
    
    while(shuttleVideo.isOpened()):
    
        hasFrame, img_frame = shuttleVideo.read()
    
    
        if not hasFrame:
            break
             
        if (int(shuttleVideo.get(1) % 5 == 0)):
                
            img_path = path + '/' + '%d.jpg' %c1
            cv2.imwrite(img_path, img_frame)
        
            # 흑백변환
            #img = cv2.imread(img_path)
            #img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #cv2.imwrite(img_path, img_gray)
            #img_gray_all.append(img_gray)
                
        
            c1 += 1

    print('done')
    

done
done
done
done


In [ ]:
video_file

'/content/drive/MyDrive/KT_AIVLE/AIVLE3rd_individual/video/jung'

In [ ]:
video_file =  video_path + '/jung'
    
shuttleVideo = cv2.VideoCapture(video_file)

while(shuttleVideo.isOpened()):
    
        hasFrame, img_frame = shuttleVideo.read()
    
    
        if not hasFrame:
            break
             
        if (int(shuttleVideo.get(1) % 5 == 0)):
                
            img_path = path + '/' + '%d.jpg' %c1
            cv2.imwrite(img_path, img_frame)
        
            # 흑백변환
            #img = cv2.imread(img_path)
            #img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #cv2.imwrite(img_path, img_gray)
            #img_gray_all.append(img_gray)
                
        
            c1 += 1

# 1. ImageDataGenerator 생성하기


1. ImageDataGenerator 생성하기
+ 모델 검증을 위해 데이터를 train:validation(8:2)로 분할합니다.
+ 모델 성능 개선을 위해 데이터 증식(Data augmentation)이 필요 시 자유롭게 설정

In [ ]:
train_datagen = ImageDataGenerator(
        validation_split = 0.2,
        #shear_range=0.2,
        horizontal_flip=True
        #rotation_range=10.,
        #width_shift_range=0.2,
        #height_shift_range=0.2
        )

train_datagen = ImageDataGenerator(rescale=1./255)

# train_genrator 생성
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    batch_size = 16,
    subset='training',
    target_size=(480, 854),
    class_mode='categorical'
  
)

# validation_generator 생성
validation_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    subset='validation',
    target_size=(480, 854),
    batch_size=16,
    class_mode='categorical' 
)

Found 0 images belonging to 4 classes.
Found 0 images belonging to 4 classes.


---

# 2. 모델 구성하기

2. Inception V3 모델을 설계하기

In [ ]:
from keras.applications.inception_v3 import InceptionV3



def get_model(weights='imagenet'):
    # create the base pre-trained model
    base_model = InceptionV3(weights=weights, include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer -- let's say we have 5 classes
    for _, dirs, _ in os.walk(train_dir):
      break
    predictions = Dense(len(dirs), activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def fine_tune_inception_layer(model):
    """After we fine-tune the dense layers, train deeper."""
    # we chose to train the top 2 inception blocks, i.e. we will freeze
    # the first 172 layers and unfreeze the rest:
    for layer in model.layers[:172]:
        layer.trainable = False
    for layer in model.layers[172:]:
        layer.trainable = True

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    model.compile(
        optimizer=SGD(lr=0.0001, momentum=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_k_categorical_accuracy'])

    return model

def train_model(model, nb_epoch, generators, callbacks=[]):
    train_generator, validation_generator = generators
    model.fit(
        train_generator,
        steps_per_epoch=10,
        validation_data=validation_generator,
        validation_steps=1,
        epochs=nb_epoch,
        callbacks=callbacks)
    return model

def main(weights_file):

    model = get_model()
    generators = get_generators()

    if weights_file is None:
        print("Training Top layers.")
        model = train_model(model, 10, generators)
    else:
        print("Loading saved model: %s." % weights_file)
        model.load_weights(weights_file)

    # Get and train the mid layers.
    model = fine_tune_inception_layer(model)
    model = train_model(model, 20, generators,
                        [checkpointer, early_stopper, tensorboard])

if __name__ == '__main__':
    weights_file = None
    main(weights_file)

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model

resnet = ResNet50(
    input_shape = IMAGE_SIZE + [3], # Making the image into 3 Channel, so concating 3.
    weights = 'imagenet', # Default weights.
    include_top = False   # 
)
x = Flatten() (resnet.output)
prediction = Dense(4, activation = 'softmax')(x)

model = Model(inputs = resnet.input, outputs = prediction)